# Clustering

In this notebook, we learn about one of the most common un-supervised learning methods: clustering. There isn't a single algorithm for clustering, but the most common one is called **k-means clustering** where $k$ refers to the number of clusters we wish to have. Note that $k$ isn't really something we can learn from the data. It's something we must specify ahead of time, and while there are some guidelines we can use to choose a reasonable value for $k$ (see the assignment), ultimately it's somewhat of a subjective choice. In fact, with un-supervised learning in general, there is a lot of subjectivity involved, making it hard to interpret results.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns

import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.cluster import KMeans

## Census Data
The data set is available from the UCI Machine Learning Repository.  The data were gathered during a US census.
<br/>Description:  https://archive.ics.uci.edu/ml/datasets/adult
<br/>Download:  https://archive.ics.uci.edu/static/public/2/adult.zip

In [ ]:
path = '../data/adult_train.csv'
census = pd.read_csv(path, sep = ",", header = 0)

# Some cleanup of the dataset
census = census.drop(columns = ['fnlwgt', 'age'])
census.columns = census.columns.str.replace("-", "_")

# Present the data
print('census shape is', census.shape)
census.head()

## Distances, Dimensions, Normalizations

The k-means clustering algorithm tries to find which rows of the dataframe are similar to each other, where similarity is based having attributes (columns) that are close to each other. To determine closeness, we use **Euclidean distance**, although we can also experiment with other distance metrics. Let's say for the sake of example we have only two columns: `education_num` and `capital_gain`. Let's grab two rows of the data, namely rows 8 and 59.

In [ ]:
ClusterCols2D = ['education_num', 'capital_gain']

two_rows = census.loc[[8, 59], ClusterCols2D]
two_rows

Let $E_8$ and $C_8$ refer to `education_num` and `capital_gain` at row with index 8, and $E_{59}$ and $C_{59}$ refer to `education_num` and `capital_gain` at row with index 59, the the Euclidean distance between the two rows is given by the following equation: 

$$D_{(8, 59)} = \sqrt{(E_8-E_{59})^2 + (C_8-C_{59})^2}$$


### Exercise (5 minutes)

- Use `numpy` to calculate the Euclidean distance between the two rows. There is more than one way of doing this.

In [ ]:
# Add Code here

# E8 - E59
Dist_edu = two_rows.loc[8, 'education_num'] - two_rows.loc[59, 'education_num']

# C8 - C59
Dist_cap =  two_rows.loc[8, 'capital_gain'] - two_rows.loc[59, 'capital_gain']

# D_8_59
Dist_edu_cap = np.sqrt(np.square(Dist_edu) + np.square(Dist_cap))

print('education_num distance: {}\ncapital_gain distance: {}\ntotal distance: {}'. format(Dist_edu, Dist_cap, Dist_edu_cap))

Notice that the Euclidian distance is dominated by the difference in the rows in the `captial_gain` column. This is because this column has a much bigger **scale** than `education_num`. So what can we do to make sure both columns can equally influence the distance? We can **normalize** the columns.

Use `numpy` to normalize the columns of the data. We will use **Z-normalization**, which is the following transformation: 

$$x_{\text{norm}} = \dfrac{x - \text{mean}(x)}{\text{std}(x)}$$

Where the mean and standard deviation are calculated on the **whole data within each column**, not just the two rows above. 

- Find the mean and standard deviations of `education_num` and `capital_gain` for the whole `census` data and use them to normalize the `two_rows`. HINT: You can calculate the mean using `.mean()`. You can get the standard deviation similarly, using the `std` method.

In [ ]:
census.mean()

In [ ]:
# Add Code here
# Find means and standard deviations
Means = census[['education_num', 'capital_gain']].mean()
print('Means are:')
print(Means.round(2))
Stds = census[['education_num', 'capital_gain']].std()
print('\nStandard Deviations are:')
print(Stds.round(2))

In [ ]:
# Add Code here
# Get normalized values
two_rows_norm = two_rows.copy()
two_rows_norm['education_num'] = (two_rows_norm['education_num'] - Means['education_num'])/Stds['education_num']
two_rows_norm['capital_gain'] = (two_rows_norm['capital_gain'] - Means['capital_gain'])/Stds['capital_gain']

# Alternate approach with sklearn
# from sklearn.preprocessing import StandardScaler 
# eduScaler = StandardScaler().fit(census[['education_num']])
# capScaler = StandardScaler().fit(census[['capital_gain']])
# two_rows_norm['education_num'] = eduScaler.transform(two_rows[['education_num']])
# two_rows_norm['capital_gain'] = capScaler.transform(two_rows[['capital_gain']])

two_rows_norm.round(2)

- Calculate the Euclidean distance of the **normalized** values of `two_rows`.

In [ ]:
# Add Code here
# Calculate Euclidean distance
Dist_edu_norm = two_rows_norm.loc[8, 'education_num'] - two_rows_norm.loc[59, 'education_num']
Dist_cap_norm =  two_rows_norm.loc[8, 'capital_gain'] - two_rows_norm.loc[59, 'capital_gain']
Dist_norm = np.sqrt(np.square(Dist_edu_norm) + np.square(Dist_cap_norm))

print('\
    normalized education_num distance: {:.3}\n\
    normalized capital_gain distance: {:.3}\n\
    normalized total distance: {:.3}'.
    format(Dist_edu_norm, Dist_cap_norm, Dist_norm))

If we had more than two columns we simply extend the formula like this:

$$D_{(i, j)} = \sqrt{(X1_i-X1_j)^2 + (X2_i-X2_j)^2 + (X3_i-X3_j)^2 + ...}$$

### End of exercise

Let's now normalize all our numeric columns in one go using `pandas`. First we get a list of the numeric columns:

In [ ]:
num_cols = census.select_dtypes(['integer', 'float']).columns
print(num_cols)

Now we use the `apply` method to apply a function to all the numeric columns at once, using `axis = 0` to say that the function applies across rows. We can either create a function ahead of time and pass it to apply, or in our case since the function is quite simple, we simply create it on the fly using the **lambda notation**.

In [ ]:
census_rescaled = census[num_cols]
census_rescaled = census_rescaled.apply(lambda x: (x - x.mean()) / x.std(), axis = 0)
census_rescaled.head().round(2)

We can create a scatter plot of the `education_num` and `capital_gain`.  These are the same two columns that we used to illustrate our previous example.

In [ ]:
sns.scatterplot(x = 'education_num', y = 'capital_gain', data = census_rescaled, alpha=0.25, size = 2, legend=False);

So for now let's start with $k=2$ and use k-means to cluster the data. We use the `KMeans` function in `sklearn`. At this point, you should be familiar with the following pattern `KMeans` and other `sklearn` functions abide by:

1. We initialize the algorithm and specify any arguments if need be. In this case, the number of clusters.
1. We call `fit` and pass it the data. This is when learning happens.
1. We call `predict` which is analogous to `transform`.  From `predict` we get the "predicted" (matched) labels back. For k-means the predictions are the **cluster assignments**. Any row will be assigned one of $k$ labels, depending on which cluster they belong to. Note that label assignments have no particular order to them: If you rerun the algorithm, what was cluster 2 last round might be called cluster 4 this round. The 2 and 4 are not important. What's important is that all the points in cluster 2 are similar to each other, and all the points in cluster 4 are similar to each other.

The above three steps are marked in the code below.

In [ ]:
n_clusters = 2 # the number of clusters (k)
ClusterCols2D = ['education_num', 'capital_gain']

X = census_rescaled[ClusterCols2D]
kmeans = KMeans(n_clusters = n_clusters) # step 1: initialize
kmeans.fit(X) # step 2, learn the clusters
census_rescaled['cluster'] = kmeans.predict(X) # step 3, assign a cluster to each row
census_rescaled.head().round(2)

We could have saved ourselves step 3 because in step 2, `kmeans.fit(X)`, the cluster labels for the training data were already computed:

In [ ]:
all(census_rescaled['cluster'] == kmeans.labels_)

It is very important to note that while the cluster assignments are integers, they have no numeric value, meaning that the numbers are just labels. Cluster 1 is not necessarily closer to cluster 2 than it is to cluster 3. In fact, reruning `KMeans` doesn't guarantee that we will retain the same order. So to be clear, we will convert the `cluster` to `category` type.  We can see the obvious clustering by using different colors for different labels. 

In [ ]:
census_rescaled['cluster'] = census_rescaled['cluster'].astype('category')
sns.scatterplot(x = 'education_num', y = 'capital_gain', hue = 'cluster', 
                alpha = 0.25, data = census_rescaled);

We can find out what the cluster centroids are and how to check the distribution of cluster assignments:

In [ ]:
print('scaled cluster centroids:')
ScaledCentroids = pd.DataFrame(kmeans.cluster_centers_, columns=ClusterCols2D)
display(ScaledCentroids)
print('\nNumber of items in each cluster:')
print(census_rescaled.cluster.value_counts())

Keep in mind that these are centroids based on the **normalized** data.  If we only have the **normalized** centroids then we would have to **un-normalize** them by running the reverse transformation to get centroids for the original data:
$$x = x_{\text{norm}} * \text{std}(x) + \text{mean}(x)$$

In [ ]:
Centroids = pd.DataFrame()
Centroids = ScaledCentroids * census[ClusterCols2D].std() + census[ClusterCols2D].mean()
Centroids

A K-means cluster center is defined as the mean location of its members.  We can also just use the original points and cluster labels to determine the cluster centers:

In [ ]:
census[ClusterCols2D].groupby(by=kmeans.labels_).mean()

### Initial Placements of Centroids
sklearn's K-means did more than we might think.  Because the initial centroid placment may determine the result, sklearn tried out multiple placements.  See:  https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html  
The number of initial placments can be set by the argument `n_init`.  We used the default value for `n_init`.  We can see what value our K-means used for `n_init`.

In [ ]:
print("K-Means was performed", kmeans.n_init, "times with different initial centroid placments")

At first glance, it's seems hard to say if there are more than 2 clusters from looking at the scatter plot above. So should we pick $k=2$ or $k>2$? The answer is that there is no easy answer. $k=2$ might be a safe choice, but has little practical value (so you're telling me that the world is made up of very rich people and everyone else, thanks! I'm so glad I hired a data scientist...). With $k>2$ we can get more refined differences, but it's hard to know where to draw the line and what sets different groups apart. And here we only have two columns and the luxury of looking at scatter plots, but as the number of features goes this becomes a harder and harder problem. Well this is the curse of un-supervised learning!

In [ ]:
n_clusters = 3 # the number of clusters (k)
ClusterCols2D = ['education_num', 'capital_gain']

X = census_rescaled[ClusterCols2D]
kmeans = KMeans(n_clusters = n_clusters, random_state = 0) # step 1: initialize
kmeans.fit(X) # step 2, learn the clusters
census_rescaled['cluster'] = kmeans.predict(X) # step 3, assign a cluster to each row
census_rescaled.head()

Let's now check the the cluster centroids and the distribution of cluster assignments.  Since our clustering is based only on `education_num` and `capital_gain`, we need to restrict the centroids to those two columns:

In [ ]:
print('cluster centroids:')
display(census[ClusterCols2D].groupby(by=kmeans.labels_).mean())
print('\nNumber of items in each cluster:')
print(census_rescaled.cluster.value_counts())

### Unsupervised (Label-Free) Learning and Matching
There is one thing to note about k-means. We said in the beginning that k-means is an un-supervised learning algorithm. This means that the data is not labeled ahead of time with clusters that we need to then learn. This means that when we use k-means to assign each row to a cluster, we have no way to **evaluate** the label assignments and determine if we did a good job. This is why we call it **un-supervised**. However, we can still do something that we usually do with **supervised** learning algorithms: we can predict for any new row of data by assigning a cluster to it. How? We simply assign the new row by first normalizing it in the same way the training data was normalized and then assigning it to the closest cluster **centroid**.  Technically, this assignment to a cluster is called **matching** and not predicting.  "Predicting" is reserved for supervised learning.  Here is an example of **matching**:

In [ ]:
# lots of school but little money
Person1 = [[2.2, -0.1]]
Person1 = pd.DataFrame(Person1, columns = ['education_num', 'capital_gain'])
print(' Person1 is matched to cluster:', kmeans.predict(Person1)[0])

# Very little school but average wealth
Person2 = [[-2.0, 0.0]]
Person2 = pd.DataFrame(Person2, columns = ['education_num', 'capital_gain'])
print(' Person2 is matched to cluster:', kmeans.predict(Person2)[0])

# Some School but very wealthy
Person3 = [[0.0, 10.]]
Person3 = pd.DataFrame(Person3, columns = ['education_num', 'capital_gain'])
print(' Person3 is matched to cluster:', kmeans.predict(Person3)[0])

Note that we had to use normalized data for matching.  The data processing steps used for the training data has to be applied to the operational (new) data before matching.

### Inertia (Measure of Clustering)
We can measure the remaining variation in the data after we cluster the data by looking at the `inertia_` attribute. This value is computed as by taking the sum of squared distances of each data point to their closest cluster center.  A smaller number means denser clusters.  

In [ ]:
print("The remaining inertia (Sum of Square Distances) is", int(kmeans.inertia_))

Using `numpy` we can manually obtain this value. It helps sometimes to "reverse engineer" such computations to get a deeper understanding and familiarity with the algorithm. Examine the code below as one example.

In [ ]:
inertia = 0

for cls in range(n_clusters):
    # find the subset of the data with cluster assignment
    census_subset_cls = census_rescaled.loc[census_rescaled['cluster'] == cls, ClusterCols2D].values
    # find the sum of the squared differences between the data and the corresponding centroid
    inertia += np.power(census_subset_cls - kmeans.cluster_centers_[cls], 2).sum()
    
print('We calculated an inertia of:', int(inertia))

We can see that we're able to get the same value as what `KMeans` returned. 

We can redraw our earlier scatter plot and color-code the points by the cluster they belong to.

In [ ]:
census_rescaled['cluster'] = census_rescaled['cluster'].astype('category')
sns.scatterplot(x = 'education_num', y = 'capital_gain', hue = 'cluster', 
                data = census_rescaled);

- What differentiates cluster 1 form 2, 1 from 3, and 2 from 3?

### Exercise (10 minutes)

- Based on the above scatter plot, which of the two features do you think is more important in determining which cluster a person belongs to?
- Return to where we called `KMeans` and change the number of clusters to $k=4$. 
- Report your findings. You can see that as $k$ increases, we have more and more comparisons to make.

In [ ]:
# Add code here
n_clusters = 4 # the number of clusters (k)
which_cols = ['education_num', 'capital_gain']

X = census_rescaled[which_cols]
kmeans = KMeans(n_clusters = n_clusters, random_state = 0) # step 1: initialize
kmeans.fit(X) # step 2, learn the clusters
census_rescaled['cluster'] = kmeans.predict(X) # step 3, assign a cluster to each row
census_rescaled.head()

In [ ]:
print('cluster centroids:')
display(census[ClusterCols2D].groupby(by=kmeans.labels_).mean())
print('\nNumber of items in each cluster:')
print(census_rescaled.cluster.value_counts(sort=False))
print("\nThe remaining inertia (Sum of Square Distances) is", kmeans.inertia_)

In [ ]:
census_rescaled['cluster'] = census_rescaled['cluster'].astype('category')
sns.scatterplot(x = 'education_num', y = 'capital_gain', hue = 'cluster', 
                data = census_rescaled);

### Higher Dimensional Clustering
- Let's keep $k=4$, but this time pass all four numeric columns to `KMeans`.
- Centroids:  What is the most obvious difference in the centroids?
- Scatter Plot:
    - Do you notice any changes to the scatter plot?
    - Can you see all the clusters?

In [ ]:
# Add code here
n_clusters = 4 # the number of clusters (k)

# Use all numeric columns!
X = census_rescaled[num_cols]
kmeans = KMeans(n_clusters = n_clusters, random_state = 0) # step 1: initialize
kmeans.fit(X) # step 2, learn the clusters
census_rescaled['cluster'] = kmeans.predict(X) # step 3, assign a cluster to each row

print('cluster centroids in original space:')
display(census[num_cols].groupby(by=kmeans.labels_).mean())

print('\nNumber of items in each cluster:')
print(census_rescaled.cluster.value_counts(sort = False))

census_rescaled['cluster'] = census_rescaled['cluster'].astype('category')
sns.scatterplot(x = 'education_num', y = 'capital_gain', hue = 'cluster', 
                data = census_rescaled);

Do you see problems interpreting this scatterplot?

Of course since we added all four numeric columns to `KMeans`, then we have to look at scatter plots of all possible combinations of those four columns: there are $4 \choose 2$ (we read that as **4 choose 2**) which is $\frac{4!}{2!2!} = 6$ possible combinations. There is a very easy way to get all the combinations using the `itertools.combinations` function.

In [ ]:
from itertools import combinations
all_pairs = list(combinations(num_cols, 2))

for pair in all_pairs:
    print(pair)

We can now plot all the possbile scatter plots from the pairs above.

In [ ]:
fig, axs = plt.subplots(nrows = 3, ncols = 2, figsize = (18, 12))
sns.scatterplot(x = all_pairs[0][0], y=all_pairs[0][1], hue = 'cluster', data = census_rescaled, ax = axs[0, 0]); #, legend = False);
sns.scatterplot(x = all_pairs[1][0], y=all_pairs[1][1], hue = 'cluster', data = census_rescaled, ax = axs[1, 0], legend = False);
sns.scatterplot(x = all_pairs[2][0], y=all_pairs[2][1], hue = 'cluster', data = census_rescaled, ax = axs[2, 0], legend = False);
sns.scatterplot(x = all_pairs[3][0], y=all_pairs[3][1], hue = 'cluster', data = census_rescaled, ax = axs[0, 1], legend = False);
sns.scatterplot(x = all_pairs[4][0], y=all_pairs[4][1], hue = 'cluster', data = census_rescaled, ax = axs[1, 1], legend = False);
sns.scatterplot(x = all_pairs[5][0], y=all_pairs[5][1], hue = 'cluster', data = census_rescaled, ax = axs[2, 1], legend = False);

- What differentiates clusters 1, 2, 3, and 4 from each other?

Let's say you are now charged with **profiling** the clusters. That means you need to come up with a brief description of each of the 4 clusters. You can rely on the above scatter plots or any other summaries you like. This is not necessarily an easy task, and the point is to show you the challenge of dealing with un-supervised learning algorithms. Imagine how much harder this would have been with a much larger $k$ or with more features in the data!

### End of exercise

## Categorical Columns
Our clusters so far only used the numeric columns in the data, but we also have a lot of categorical columns and we should be using them too.  We can look at the first few rows of the dataframe to identify the categorical columns.

In [ ]:
census.head()

### Categorical Data Pre-Processing
To reduce the complexity of our categorical values, we will clean the data in the following way:
1. We relpace the any hyphen with underscore in the column names.
1. In the `income` column, we replace `<=` with `lt_` and `>` with `gt_`

Can `DataFrame.replace("<=", "lt_")` assist us in replacing the items?  `DataFrame.replace("<=", "lt_")` would replace all values in the DataFrame that are "<=".  But there are no such values.  Instead there are values like "<=50K".  What we want to do is:
1. Find all values that contain the sub-string "<="
2. Modify the found values by substituting "<=" with a replacement string, namely "lt_"

### Regex
Let us ponder `DataFrame.replace("<=", "lt_", regex = True)`
- What does it mean that regex is True?
- What are regular expressions, aka regex?  

Regex is a powerful language for recognizing and modifying text.

In [ ]:
census["income"] = census["income"].replace("<=", "lt_", regex = True)
census["income"] = census["income"].replace(">", "gt_", regex = True)

census["income"].value_counts()

### Exercise:  (5 minutes)

Let's get a list of all the categorical columns in the data:

In [ ]:
cat_vars = census.select_dtypes('object').columns.to_list()
cat_vars

- For each of the categorical columns in `cat_vars` do the following:
  - Find all the values that have hypens and replace them with underscores.
  - Convert all the strings to lower case.

In [ ]:
# Add code here
# The following line is regex and is fast
census[cat_vars] = census[cat_vars].replace(to_replace='-', value='_', regex=True)
# The following line is pandas and may take upto a minute
census[cat_vars] = census[cat_vars].apply(lambda column: column.str.lower(), axis=1)

- Show the top 5 rows of the data to make sure your transformations worked.

In [ ]:
# Add code here
census.head()

### End of exercise

### Euclidean Distance with Categorical Variables
As we saw earlier k-means clustering relies on Euclidean distance to measure the similarity between the rows. So how do you measure Euclidean distance when you have categorical data? The answer is  one-hot encoding. The quick and easy way to do this is using `pd.get_dummies` function. One-hot encoded (binary) features are also called **dummy variables**, which explains why the function is named `get_dummies`.

In [ ]:
cat_vars = census.select_dtypes('object').columns.to_list()
census_onehot = pd.get_dummies(census[cat_vars])
census_onehot.head()

Let's combine our standardized numeric features and our one-hot-encoded categorical features into one data frame.

In [ ]:
census_featurized = pd.concat([census_rescaled[num_cols], census_onehot], axis = 1)

We're going to train K-Means with $k=5$ just to use a bigger number.

In [ ]:
kmeans = KMeans(n_clusters = 5, random_state = 0)
kmeans.fit(census_featurized);

Instead of adding the clusters as a new column to the featurized data, we add them to the original data. This way we can get summaries on the original (non-normalized) numeric features which makes it easier to interpret results.

In [ ]:
census['cluster'] = kmeans.labels_

Here's a list of the clusters and their size.

In [ ]:
census[['cluster']].value_counts(sort=False)

Since we also used one-hot encoded categorical variables, the averages of only the numeric columns would not be the complete centroids.  We need to add the results of the one-hot encoding.  It's definitely easier to compare cluster centroids then to look at $n \choose 2$ scatter plots when $n$ is large.  Here's the average of the original numeric features grouped by each cluster.   

In [ ]:
censusNumeric = pd.concat([census, census_onehot], axis = 1)
censusNumeric.groupby('cluster').mean().round(2)

In addition to comparing averages, it's also important to compare the variability within each cluster. However, if we want to compare variability across features, we need to use standardized features otherwise features on a larger scale will always have more variability.

In [ ]:
censusScaledNumeric = pd.concat([census_rescaled[num_cols], census_onehot], axis = 1)
censusScaledNumeric.groupby(kmeans.labels_).std()

How do we check how clusters compare across the categories of a single categorical variable? One way is to look at two-way tables. We can use `pd.crosstab` for that.

In [ ]:
pd.crosstab(census['cluster'], census['workclass'])

### Exercise (4 minutes)

- How can we make the above information more useful. We can turn the counts into percentages by cluster. Find how you can use the `normalize` argument, to turn the counts into percentages. Note that `normalize` here has nothing to do with Z-normalization we learned earlier.

In [ ]:
# Add code here
pd.crosstab(census['cluster'], census['workclass'], normalize=True, margins=True)

- Even more useful would be to display the above table with the percentages as a **heat map**, so that we can quickly compare the distribution of the clusters across different occupations. Turn the above table into a heat map using `seaborn`.

In [ ]:
# Add code here
sns.heatmap(pd.crosstab(census['cluster'], census['workclass'], normalize=True))

- Does anything particularly stand out? Can you refine your profile of each of the clusters based on what you see? It might help to also try other categorical features like `education`, `marital_status`, or `income`.

### End of exercise

There's so much more to say about clustering. Here are two examples:

- We could try to solve the problem of clustering when we have categorical data by defining a distance function that works for categorical data. 
- We could try to find a way to cluster the data **hierarchically**, so that we depend less on a specific choice of $k$. Instead we narrow our choice later by choosing the level of hierarchy we want to stop at. 

The topic of clustering can be its own course, and admittedly the `KMeans` algorithm in `sklearn` is rather limited. So we should also explore other options.

# Assignment

Assignment will be posted tomorrow afternoon 